In [ ]:
#t1
sample1 = data1.sample(105, random_state=56)
sample1.isna().sum()
sample1['total_bedrooms'] = sample1['total_bedrooms'].fillna(sample1['total_bedrooms']).mean()
sample1.isna().sum()
sample1.head()
sample1_ohe = pd.get_dummies(sample1, columns=['ocean_proximity'])
X = sample1_ohe.drop('median_house_value', axis=1)
y = sample1_ohe['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.13, random_state=12)
dt = DecisionTreeRegressor(max_depth=10, random_state=12)
dt.fit(X_train, y_train)
dt.score(X_train, y_train)
dt.score(X_test, y_test)


In [ ]:
sample2 = data1.sample(500, random_state=56)
sample2.isna().sum()
sample2['total_bedrooms'] = sample2['total_bedrooms'].fillna(sample2['total_bedrooms']).mean()
sample2.isna().sum()
sample2.head()
sample2_ohe = pd.get_dummies(sample2, columns=['ocean_proximity'])
X = sample2_ohe.drop('median_house_value', axis=1)
y = sample2_ohe['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.13, random_state=12)
dtr2 = DecisionTreeRegressor(max_depth=10, random_state=12)
dtr2.fit(X_train, y_train)
print(f"Train accuracy: {dtr2.score(X_train, y_train): .4f}")
print(f"Test accuracy: {dtr2.score(X_test, y_test): .4f}")
dtr2.feature_importances_
dtr2.feature_names_in_
pd.Series(data = dtr2.feature_importances_, index=dtr2.feature_names_in_).sort_values(ascending=False)

<!-- - Visualize the data using a scatter plot (x, y) using code below


- Apply DBSCAN clustering and identify core
- Count how many points fall into noise vs clusters. -->

X = data2[['x','y']]

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.2, min_samples=5)
clusters = dbscan.fit_predict(X)
data2
data2['cluster'] = clusters
data2

plt.figure(figsize=(8, 6))
unique_labels = np.unique(data2['cluster'])
colors = {-1: 'red', 0: 'blue', 1: 'green', 2: 'purple'}  # Extend as needed

for label in unique_labels:
    mask = data2['cluster'] == label
    cluster_name = "Noise" if label == -1 else f"Cluster {label}"
    plt.scatter(data2[mask]['x'], data2[mask]['y'],
                c=colors.get(label, 'gray'), s=10, label=cluster_name)
plt.title("DBSCAN Clustering with Core and Border Points")
plt.legend(loc='upper left')
plt.show()

n_clusters = len(set(data2['cluster'])) - (1 if -1 in data2['cluster'].values else 0)
n_noise = np.sum(data2['cluster'] == -1)

print(f"Number of clusters: {n_clusters}")
print(f"Number of noise: {n_noise}")

In [ ]:
# Implement Agglomerative Clustering for data 2 given in task 4
from sklearn.cluster import AgglomerativeClustering
agg = AgglomerativeClustering().fit(X)
ac_predictions=agg.fit_predict(X)
ac_predictions
agg.labels_

#option1:use dendogram (ask if we need to use dendogram)
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt

Z = sch.linkage(X, method='ward')  # You can try other linkage methods like 'average', 'single', etc.

plt.figure(figsize=(10, 7))
sch.dendrogram(Z)
plt.title("Dendrogram")
plt.xlabel("Samples")
plt.ylabel("Distance")
plt.show()

# The silhouette score measures how well each sample fits within its cluster. A higher silhouette score indicates that the clusters are well-defined.

# Silhouette score ranges from -1 to +1.

# - A score close to +1 indicates that the samples are well clustered.
# - A score close to 0 indicates overlapping clusters.
# - A negative score suggests that samples might have been assigned to the wrong clusters.

#Option 2: Use Silhouette Score
#Use silhouette score to find the best number of clusters:
from sklearn.metrics import silhouette_score
sil_score = silhouette_score(X, ac_predictions)
print(f"Silhouette Score: {sil_score}")


# The silhouette score measures how well each sample fits within its cluster. A higher silhouette score indicates that the clusters are well-defined.

# Silhouette score ranges from -1 to +1.

# - A score close to +1 indicates that the samples are well clustered.
# - A score close to 0 indicates overlapping clusters.
# - A negative score suggests that samples might have been assigned to the wrong clusters.



# Silhouette Score: 0.31444833716290493
# The above silhouette score indicates that the clusters are not well-defined. And may overlap each other. Therefore, the cluster doesn't capture the underlying structure in the data.


In [ ]:
# Implement K-means clustering for data used in task
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)  # Set a random_state for reproducibility
clusters2 = kmeans.fit(data2)  # Fit the KMeans model to the data

clusters2.predict(data2)
kmeans.cluster_centers_.shape
from scipy.cluster.hierarchy import dendrogram, linkage

# Select only 'x' and 'y' for clustering
X = data2[['x', 'y']].values

link = linkage(X[:15,:], method='ward')  # X - data used

plt.figure(figsize=(12, 6))
dendrogram(link,
           orientation='top',
           distance_sort='ascending',
           show_leaf_counts=False,
           truncate_mode='level')
plt.title('Hierarchical Clustering Dendrogram')
plt.show()

#extra last
# - Analyze: Why is DBSCAN more effective for this kind of spatial distribution compared to K-means?
# 1. DBSCAN clusters by density, not distance to a centroid:  
#     - DBSCAN grows clusters by "chaining" together points that lie in high density
# 2. No assumption of COnvex or Spherical shape: 
#     - KMeans optimizes around a single center and creates convex (ball-shaped) clusters. 
#     - DBSCAN links nearby points, regardless of overall shape, so it follws the curving geometry of each ring
# 3. Automaticaly detects number of clusters: 
#     - DBSCAN discovers number of clusters based on density thresholds.




In [ ]:
# Based on the hierarchical clustering dendrogram generated using below code, if you decide to form 3 distinct clusters, identify and list the data points assigned to each of the resulting clusters
from scipy.cluster.hierarchy import dendrogram, linkage

# Select only 'x' and 'y' for clustering
X = data2[['x', 'y']].values

link = linkage(X[:15,:], method='ward')  # X - data used

plt.figure(figsize=(12, 6))
dendrogram(link,
           orientation='top',
           distance_sort='ascending',
           show_leaf_counts=False,
           truncate_mode='level')
plt.title('Hierarchical Clustering Dendrogram')
plt.show()

# If we cut the tree to form 3 clusters, the points in each are:
# - Cluster 1 (orange): 1, 3, 10
# - Cluster 2 (green): 13, 8, 4, 12, 14
# - Cluster 3 (red): 6, 0, 11, 7, 9, 2, 5



In [ ]:
#oneht
# Select the features and target variable
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
target = 'Survived'
X = titanic_data[features]
y = titanic_data[target]

numeric_features = ['Age', 'Fare']
categorical_features = ['Pclass', 'Sex', 'Embarked']


X[categorical_features]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

X = preprocessor.fit_transform(X)
X

